# Find Course or Room Chunks

In [10]:
#!/usr/bin/env python
# coding: utf8
"""Example of training an additional entity type
This script shows how to add a new entity type to an existing pretrained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.
The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.
After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.
For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities
Compatible with: spaCy v2.1.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# new entity label
CLASS_LABEL = "CLASS"
ROOM_LABEL = "ROOM"
UNKNOWN_LABEL = "UNKNOWN"

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
TRAIN_DATA = [
    # Unknown
    (
        "Where is 151?",
        {"entities": [(9, 12, UNKNOWN_LABEL)]},
    ),
    (
        "How do I get to 151?",
        {"entities": [(16, 19, UNKNOWN_LABEL)]},
    ),
    (
        "I can't find 112?",
        {"entities": [(13, 16, UNKNOWN_LABEL)]},
    ),
    
    # Classes
    (
        "Where is CS 151?",
        {"entities": [(9, 15, CLASS_LABEL)]},
    ),
    (
        "Where is CS 111 G?",
        {"entities": [(9, 17, CLASS_LABEL)]},
    ),
    (
        "What room is CS Principles in",
        {"entities": [(13, 26, CLASS_LABEL)]}
    ),
    (
        "What room is Python Programming 1",
        {"entities": [(13,32,CLASS_LABEL)]}
    ),
    (
        "How do I get to R Programming",
        {"entities": [(16,29,CLASS_LABEL)]}
    ),
    (
        "How do I get to Intro to Computer Sci",
        {"entities": [(16,37,CLASS_LABEL)]}
    ),
    
    (
        "How do I get to Intro to Computer Science",
        {"entities": [(16,41,CLASS_LABEL)]}
    ),
    (
        "Compilers",
        {"entities": [(0, 9, CLASS_LABEL)]},
    ),
    
    (
        "I have databases today but I do not know where it is", 
        {"entities": [(7, 16, CLASS_LABEL)]}
    ),
    (
        "I have databases 1 today but I do not know where it is", 
        {"entities": [(7, 18, CLASS_LABEL)]}
    ),
    (
        "Is CS 272 in room 107?",
        {"entities": [(3, 9, CLASS_LABEL)]},
    ),
    (
        "Is algorithms are there?", 
        {"entities": [(3, 13, CLASS_LABEL)]}
    ),
        (
        "Is CS 151 in science hall?",
        {"entities": [(3, 9, CLASS_LABEL)]},
    ),
    (
        "How do I get to CS 111 G?",
        {"entities": [(16, 24, CLASS_LABEL)]},
    ),
    
    # Rooms
    (
        "Where is room 118",
        {"entities": [(9,17,ROOM_LABEL)]}
    ),
    (
        "Where is room 118",
        {"entities": [(9,17,ROOM_LABEL)]}
    ),
    (
        "Where is room 225",
        {"entities": [(9,17,ROOM_LABEL)]}
    ),
    (
        "Where is the computer science lab",
        {"entities": [(13,33,ROOM_LABEL)]}
    ),
    (
        "Where is the lab",
        {"entities": [(13,16,ROOM_LABEL)]}
    ),
    (
        "Where is the CS Lounge?",
        {"entities": [(13,22,ROOM_LABEL)]}
    ),
    (
        "Where is the student lounge?",
        {"entities": [(13,28,ROOM_LABEL)]}
    ),
    (
        "Where is the lounge?",
        {"entities": [(13,19,ROOM_LABEL)]}
    ),
    (
        "Where is the math department?",
        {"entities": [(13,28,ROOM_LABEL)]}
    ),

    (
        "Where is the psychology department?",
        {"entities": [(13, 34, ROOM_LABEL)]}
    ),
    
    (
        "How do I get to the lab",
        {"entities": [(20,23,ROOM_LABEL)]}
    ),
    (
        "How do I get to the bathroom",
        {"entities": [(20, 28, ROOM_LABEL)]},
    ),
    
    (
        "How do I get to the math department", 
        {"entities": [(20, 35, ROOM_LABEL)]}
    ),
    (
        "How do I get to the psychology department", 
        {"entities": [(20, 41, ROOM_LABEL)]}
    ),
    (
        "How do I get to science hall 112?",
        {"entities": [(16, 32, ROOM_LABEL)]},
    ),
    (
        "How do I get to s h 105?", 
        {"entities": [(16, 24, ROOM_LABEL)]}
    ),
    
]




In [13]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(model=None, new_model_name="class", output_dir=None, n_iter=30):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    ner.add_label(UNKNOWN_LABEL)  # add new entity label to entity recognizer
    ner.add_label(CLASS_LABEL)  # add new entity label to entity recognizer
    ner.add_label(ROOM_LABEL)  # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses", losses)

    # test the trained model
    test_text = "Where is Introduction to Philosophy?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "Is CS 448 offered next fall?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "Is datamining a class here?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    test_text = "Where is room 103?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "How do I get to the bathroom?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)
        
    test_text = "Which way is the psychology department?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

        
    test_text = "Which way is 118?"
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        # Check the classes have loaded back consistently
        assert nlp2.get_pipe("ner").move_names == move_names
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)




In [15]:
main()

Created blank 'en' model
Losses {'ner': 94.97624290819294}
Losses {'ner': 53.62186853332088}
Losses {'ner': 54.33504400929686}
Losses {'ner': 55.613201058317756}
Losses {'ner': 40.126727742594646}
Losses {'ner': 40.178867800950556}
Losses {'ner': 44.500372033239934}
Losses {'ner': 35.695940612073606}
Losses {'ner': 27.285165922131593}
Losses {'ner': 20.61849937035448}
Losses {'ner': 19.269689037325442}
Losses {'ner': 23.721540624226897}
Losses {'ner': 16.250851927049627}
Losses {'ner': 27.763792361107637}
Losses {'ner': 9.876086554191382}
Losses {'ner': 12.97845721221638}
Losses {'ner': 14.37631652442872}
Losses {'ner': 6.497860648718981}
Losses {'ner': 1.5453108490815268}
Losses {'ner': 7.373509559758265}
Losses {'ner': 0.0416104879806762}
Losses {'ner': 0.44741480285722407}
Losses {'ner': 0.10912281868359325}
Losses {'ner': 1.4791420789666565}
Losses {'ner': 0.9497626953980899}
Losses {'ner': 0.7951705155322918}
Losses {'ner': 0.0012031965424230571}
Losses {'ner': 0.01121254801059658